# In this notebook, we gain an intuition of training transformers and try to see it's pros and cons 

In [5]:
import tiktoken
import torch
import torch.nn as nn
import numpy as np
import time

# Create the models

import math
import torch.nn.functional as F


### Create a Next character level prediction dataset

In [11]:
class NextCharDataset(torch.utils.data.Dataset):

    def __init__(self, dataset_path , context_length):
        self.dataset_path = 
        self.context_length = 
        
        with open(self.dataset_path,"r") as f:
            self.raw_text = f.read()

        self.tokens = 
        
        # Remap the tokens
        
        remapping = np.arange(len(set(self.tokens))).tolist()
        self.mapping = dict(zip(list(set(self.tokens)),remapping))
        self.inverse_mapping = dict(zip(remapping,list(set(self.tokens))))
        
        self.vocab_size = len(set(self.tokens))
        
        

    def __len__(self):
        return 

    def __getitem__(self, idx):

        

        return torch.tensor(x,dtype=torch.long), torch.tensor(y,dtype=torch.long)

### Create a next Token prediction dataset

In [2]:
class NextTokenDataset(torch.utils.data.Dataset):

    def __init__(self, dataset_path , context_length):
        self.dataset_path = dataset_path
        
        
        

    def __len__(self):
        return 

    def __getitem__(self, idx):



        return torch.tensor(x,dtype=torch.long), torch.tensor(y,dtype=torch.long)

In [13]:
def create_datasets(dataset_class, dataset_path, context_length):
    full_dataset = dataset_class(dataset_path, context_length)
    train_dataset ,  test_dataset = torch.utils.data.random_split(full_dataset, [0.99,0.01])
    
    return train_dataset, test_dataset



In [14]:
train_dataset, test_dataset = create_datasets(NextCharDataset,"hymns.txt", 100)
batch_size = 100
shuffle = True
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size, shuffle=shuffle)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size, shuffle=shuffle)

In [4]:

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self,ntoken, ninp, nhid, nlayers, dropout=0.0):
        super(RNNModel, self).__init__()
        self.ntoken = ntoken
        self.drop = nn.Dropout(dropout)
        self.input_emb = nn.Embedding(ntoken, ninp)
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout,batch_first=True)
        self.output_layer = nn.Linear(nhid, ntoken)
        self.init_weights()

        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.input_emb.weight, -initrange, initrange)
        nn.init.zeros_(self.output_layer.bias)
        nn.init.uniform_(self.output_layer.weight, -initrange, initrange)

    def forward(self, x):
        bs,context = x.size()
        hidden = self.init_hidden(bs)
        emb = self.drop(self.input_emb(x))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.output_layer(output)

        decoded = F.log_softmax(decoded, dim=1)

        return decoded

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                weight.new_zeros(self.nlayers, bsz, self.nhid))


### Positionnal Embedding

In [ ]:
# Temporarily leave PositionalEncoding module here. Will be moved somewhere else.
class PositionalEncoding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens in the sequence.
        The positional encodings have the same dimension as the embeddings, so that the two can be summed.
        Here, we use sine and cosine functions of different frequencies.
    .. math:
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    Examples:
        >>> pos_encoder = PositionalEncoding(d_model)
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        Examples:
            >>> output = pos_encoder(x)
        """

        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

### Build your transformer model

In [5]:
class TransformerModel(nn.Transformer):
    """Container module with an encoder, a recurrent or transformer module, and a decoder."""

    def __init__(self,):
        super(TransformerModel, self).__init__()
        self.model_type = 
        self.src_mask = 
        self.pos_encoder = 

        self.input_emb = 
        self.ninp = 
        self.output_layer = 

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        return torch.log(torch.tril(torch.ones(sz,sz)))

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.input_emb.weight, -initrange, initrange)
        nn.init.zeros_(self.output_layer.bias)
        nn.init.uniform_(self.output_layer.weight, -initrange, initrange)

    def forward(self, src, has_mask=True):

        device = src.device

        mask = self._generate_square_subsequent_mask(len(src)).to(device)
        self.src_mask = mask


        
        return F.log_softmax(output, dim=-1)

In [7]:
ntoken = train_dataset.dataset.vocab_size
nembd = 64
nhead = 4
nlayer = 5
feedforward_nlayer = 50
lstm_nlayer = 5
lstm_nhid = 50

transformer_decoder = TransformerModel(ntoken,nembd,nhead ,feedforward_nlayer,nlayer )
out = transformer_decoder(torch.tensor([[10,0,5,3,6,20,5]]))
out.shape 
lstm_model = RNNModel(ntoken, nembd, lstm_nhid, lstm_nlayer, dropout=0.50).to("cuda")

/home/facto22020/anaconda3/envs/phd/lib/python3.10/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


###  Write the training loop for the transformer and the recurrent neural network

In [ ]:
model = transformer_decoder.to("cuda")
# model = lstm_model.to("cuda")
# init optimizer

weight_decay = 0.01
epochs = 150


# training loop
best_loss = None
step = 0

#for name, model , lr in zip([ "lstm"],[lstm_model],[5e-2]):
for name, model , lr in zip([ "transformer","lstm"],[transformer_decoder, lstm_model],[5e-3,5e-2]):
    
    final_lr = 5e-5
    gamma = (final_lr / lr) ** (2.0 / epochs)
    

#TO DO 

    
    print(f"--"*89)
    print(f"Model {name} | lr {lr}")
    print("--" * 89)
    
    for epoch in range(epochs):
        t0 = time.time()
        #TO DO


        t1 = time.time()
        print(f"Epoch {epoch + 1} | Loss {loss.item():.4f} | Step time {(t1 - t0) * 1000:.2f}ms | Current LR {scheduler.get_last_lr()[0]:.6f}")

        #print(f"Epoch {epoch} | loss {loss.item():.4f} | step time {(t1-t0)*1000:.2f}ms")
    print(f"Model {name} | loss {loss.item():.4f} | step time {(t1-t0)*1000:.2f}ms")

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Model transformer | lr 0.005
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1 | Loss 2.1362 | Step time 2185.15ms | Current LR 0.004702
Epoch 2 | Loss 2.0965 | Step time 2087.49ms | Current LR 0.004422
Epoch 3 | Loss 2.0929 | Step time 2075.01ms | Current LR 0.004159
Epoch 4 | Loss 2.0990 | Step time 2076.81ms | Current LR 0.003911
Epoch 5 | Loss 2.0709 | Step time 2087.96ms | Current LR 0.003678
Epoch 6 | Loss 2.0771 | Step time 2084.16ms | Current LR 0.003459
Epoch 7 | Loss 2.0994 | Step time 2091.22ms | Current LR 0.003253
Epoch 8 | Loss 2.0236 | Step time 2138.68ms | Current LR 0.003059
Epoch 9 | Loss 2.0812 | Step time 2114.00ms | Current LR 0.002877
Epoch 10 | Loss 2.0

### Write the generation code

In [10]:
@torch.no_grad()
def generate(model, idx, max_new_tokens, context, temperature=1.0, do_sample=False, top_k=None):
    """
    Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
    the sequence max_new_tokens times, feeding the predictions back into the model each time.
    Most likely you'll want to make sure to be in model.eval() mode of operation for this.
    """
    block_size = context
    for _ in range(max_new_tokens):
        # if the sequence context is growing too long we must crop it at block_size
        idx_cond = idx if idx.size(1) <= block_size else idx[:, -block_size:]
        # forward the model to get the logits for the index in the sequence
        logits= model(idx_cond)
        # pluck the logits at the final step and scale by desired temperature

        logits = logits[:, -1, :] / temperature
        # optionally crop the logits to only the top k options
        if top_k is not None:
            v, _ = torch.topk(logits, top_k)
            logits[logits < v[:, [-1]]] = -float('Inf')
        # apply softmax to convert logits to (normalized) probabilities
        probs = F.softmax(logits, dim=-1)
        # either sample from the distribution or take the most likely element
        if do_sample:
            idx_next = torch.multinomial(probs, num_samples=1)
        else:
            _, idx_next = torch.topk(probs, k=1, dim=-1)
        # append sampled index to the running sequence and continue
        idx = torch.cat((idx, idx_next), dim=1)

    return idx

### Generate

In [13]:
prompt = """ vive """

prompt_tokens = train_dataset.dataset.tokeniser.encode(prompt)
tokens = [train_dataset.dataset.mapping[i] for i in prompt_tokens]
tokens = torch.tensor([tokens],dtype=torch.long)

for name, model in zip(["transformer", "lstm"],[transformer_decoder, lstm_model]):
    generated = generate(model,tokens.to("cuda"),1000,context=10)
    response = [train_dataset.dataset.inverse_mapping[i] for i in generated.cpu().numpy().squeeze().tolist()]
    response = train_dataset.dataset.tokeniser.decode(response)
    print("=="*50)
    print(f"Name : {name}")
    print("=="*50)
    print(response)
                      
                      
                      

Name : transformer
 vive 3)

Salut Ô terre.

Et que nous avons, mon pays, mon pays, mon pays, mon pays, et on pouvait entendre
Parmiers de la liberté
Et la paix, et tu as mon pays
Tu espoir de la liberté, et la paix, mon pays, et la paix, mon amour, mon pays, mon pays, mon pays, mon pays, et de la patrie de la liberté, mon pays, et de la paix et de la liberté et de la paix.

Loyez-en témoin ! Soyez-en témoin ! Soyez-en témoin ! Soyez-en témoin ! Soyez-en témoin ! Soyez vos bataillons,
La patrie
Et tous nos sillons, mon amour, mon pays, et la paix, et la paix, mon pays, et de la paix
Et la liberté :
« Au grand peuple argentin, mon pays, et la paix.
Et que je sortirais.

Oh ! Soyez-en témoin !

Aux armes, mon pays, mon pays, et de la liberté, mon pays
Tu as mon pays, mon pays, mon pays, mon amour, mon pays, et de la paix et de la liberté
Et que le monde.
Si nous avons, mon cœur
Et ils fierté, et de la paix, mon pays
Et nous serons de la liberté
De nous a unis glorieux, et une étoilée
Et 